In [35]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# 드라이버 설정
driver = webdriver.Chrome()
url = "https://www.dongwonfnb.com/services/Product/Product_List"
driver.get(url)

# 첫 로딩 대기
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "dl.prodMenu"))
)

page_num = 1

while True:
    
    print(f"📄 현재 페이지: {page_num}")
    time.sleep(1)

    try:
        pagination = driver.find_element(By.CLASS_NAME, "boardPage")
        li_list = pagination.find_elements(By.TAG_NAME, "li")

        # 현재 페이지 인덱스 찾기
        current_idx = -1
        for i, li in enumerate(li_list):
            if "on" in li.get_attribute("class"):
                current_idx = i
                break

        next_found = False

        # 다음 숫자 페이지가 있으면 클릭
        if 0 <= current_idx < len(li_list) - 1:
            next_li = li_list[current_idx + 1]
            if "btn_next" not in next_li.get_attribute("class"):
                next_li.find_element(By.TAG_NAME, "a").click()
                page_num += 1
                next_found = True
                continue

        # 다음 페이지 세트 (btn_next) 클릭
        if not next_found:
            try:
                prev_page = page_num

                next_btn = driver.find_element(By.CSS_SELECTOR, "ul.boardNav > li > a.btn_next")
                driver.execute_script("arguments[0].click();", next_btn)
                time.sleep(2)  # 페이지 전환 대기

                # 새로운 페이지 번호 체크
                pagination = driver.find_element(By.CLASS_NAME, "boardPage")
                li_list = pagination.find_elements(By.TAG_NAME, "li")

                for i, li in enumerate(li_list):
                    if "on" in li.get_attribute("class"):
                        page_num = int(li.text.strip())
                        break

                if page_num == prev_page:
                    print("✅ 마지막 페이지 도달 (btn_next 클릭해도 변화 없음).")
                    break

                print("➡️ 다음 버튼 클릭 (페이지 세트 이동)")
                continue
            except Exception as e:
                print(f"❗ 다음 버튼 클릭 중 에러: {e}")
                break

    except Exception as e:
        print(f"❗ 페이지 이동 중 에러 발생: {e}")
        break

driver.quit()
print("✅ 페이지 이동 테스트 종료.")


📄 현재 페이지: 1
📄 현재 페이지: 2
📄 현재 페이지: 3
📄 현재 페이지: 4
📄 현재 페이지: 5
📄 현재 페이지: 6
📄 현재 페이지: 7
📄 현재 페이지: 8
📄 현재 페이지: 9
📄 현재 페이지: 10
➡️ 다음 버튼 클릭 (페이지 세트 이동)
📄 현재 페이지: 11
📄 현재 페이지: 12
📄 현재 페이지: 13
📄 현재 페이지: 14
✅ 마지막 페이지 도달 (btn_next 클릭해도 변화 없음).
✅ 페이지 이동 테스트 종료.


In [37]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# 드라이버 설정
driver = webdriver.Chrome()
url = "https://www.dongwonfnb.com/services/Product/Product_List"
driver.get(url)

# 결과 저장 리스트
results = []

# 첫 로딩 대기
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "dl.prodMenu"))
)

page_num = 1

while True:
    print(f"📄 현재 페이지: {page_num}")
    time.sleep(2)

    # 제품 블록들 수집
    try:
        blocks = driver.find_elements(By.CSS_SELECTOR, "dl.prodMenu")
        for block in blocks:
            try:
                product_name = block.find_element(By.TAG_NAME, "dt").get_attribute("textContent").strip()

                tag_wrap = block.find_elements(By.CSS_SELECTOR, "div.prodTag_wrap > span")
                tags = [t.get_attribute("textContent").strip() for t in tag_wrap]

                results.append({
                    "제품명": product_name,
                    "태그": ", ".join(tags) if tags else ""
                })
            except Exception as e:
                print(f"❗ 제품 정보 파싱 에러: {e}")
    except Exception as e:
        print(f"❗ 블록 수집 에러: {e}")
        break

    # 페이지 이동
    try:
        pagination = driver.find_element(By.CLASS_NAME, "boardPage")
        li_list = pagination.find_elements(By.TAG_NAME, "li")

        # 현재 페이지 인덱스 확인
        current_idx = -1
        for i, li in enumerate(li_list):
            if "on" in li.get_attribute("class"):
                current_idx = i
                break

        next_found = False

        # 다음 숫자 페이지 클릭
        if 0 <= current_idx < len(li_list) - 1:
            next_li = li_list[current_idx + 1]
            if "btn_next" not in next_li.get_attribute("class"):
                next_li.find_element(By.TAG_NAME, "a").click()
                page_num += 1
                continue

        # 다음 세트 페이지 (btn_next) 클릭
        prev_page = page_num
        next_btn = driver.find_element(By.CSS_SELECTOR, "ul.boardNav > li > a.btn_next")
        driver.execute_script("arguments[0].click();", next_btn)
        time.sleep(2)

        # 새 페이지 번호 업데이트
        pagination = driver.find_element(By.CLASS_NAME, "boardPage")
        li_list = pagination.find_elements(By.TAG_NAME, "li")
        for i, li in enumerate(li_list):
            if "on" in li.get_attribute("class"):
                page_num = int(li.text.strip())
                break

        if page_num == prev_page:
            print("✅ 마지막 페이지 도달 (btn_next 클릭해도 이동 없음)")
            break

    except Exception as e:
        print(f"❗ 페이지 이동 중 에러 발생: {e}")
        break

driver.quit()

# CSV 저장
df = pd.DataFrame(results)
df.to_csv("dongwonfnb_product_list_with_tags.csv", index=False, encoding="utf-8-sig")
print("✅ 전체 제품 정보 저장 완료. 총 수집 개수:", len(df))


📄 현재 페이지: 1
📄 현재 페이지: 2
📄 현재 페이지: 3
📄 현재 페이지: 4
📄 현재 페이지: 5
📄 현재 페이지: 6
📄 현재 페이지: 7
📄 현재 페이지: 8
📄 현재 페이지: 9
📄 현재 페이지: 10
📄 현재 페이지: 11
📄 현재 페이지: 12
📄 현재 페이지: 13
📄 현재 페이지: 14
✅ 마지막 페이지 도달 (btn_next 클릭해도 이동 없음)
✅ 전체 제품 정보 저장 완료. 총 수집 개수: 335


In [ ]:
# dongwonfnb_scraper_final.py

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# ===== 드라이버 설정 =====
driver = webdriver.Chrome()
driver.get("https://www.dongwonfnb.com/services/Product/Product_List")
wait = WebDriverWait(driver, 10)
time.sleep(3)

results = []

# ===== 1차 카테고리 수집 =====
first_select = Select(wait.until(EC.presence_of_element_located((By.ID, "selCategoryD1"))))
first_options = [opt.text for opt in first_select.options if opt.text != "1차분류"]

for first in first_options:
    # fresh select 객체로 1차 카테고리 선택
    first_select = Select(driver.find_element(By.ID, "selCategoryD1"))
    first_select.select_by_visible_text(first)
    time.sleep(2)

    # ===== 2차 카테고리 수집 =====
    second_select = Select(driver.find_element(By.ID, "selCategoryD2"))
    second_options = [opt.text for opt in second_select.options if opt.text != "2차분류"]

    for second in second_options:
        # fresh select 객체로 2차 카테고리 선택
        second_select = Select(driver.find_element(By.ID, "selCategoryD2"))
        second_select.select_by_visible_text(second)
        time.sleep(1)

        # ✅ "조회" 버튼 클릭
        search_btn = driver.find_element(By.CLASS_NAME, "ico_right")
        search_btn.click()
        time.sleep(2)

        # ===== 페이지 루프 시작 =====
        page_num = 1
        while True:
            print(f"📄 {first} > {second} > 페이지 {page_num}")
            time.sleep(1)

            items = driver.find_elements(By.CSS_SELECTOR, "dl.prodMenu > dt")
            for item in items:
                results.append({
                    "1차카테고리": first,
                    "2차카테고리": second,
                    "메뉴명": item.get_attribute("textContent").strip()
                })

            # 다음 페이지 버튼 탐색
            try:
                pagination = driver.find_element(By.CLASS_NAME, "boardPage")
                li_list = pagination.find_elements(By.TAG_NAME, "li")

                current_idx = -1
                for i, li in enumerate(li_list):
                    if "on" in li.get_attribute("class"):
                        current_idx = i
                        break

                next_found = False
                if 0 <= current_idx < len(li_list) - 1:
                    next_li = li_list[current_idx + 1]
                    if "btn_next" not in next_li.get_attribute("class"):
                        next_li.find_element(By.TAG_NAME, "a").click()
                        page_num += 1
                        next_found = True
                        continue

                # 다음 세트 버튼 처리
                if not next_found:
                    try:
                        prev_page = page_num
                        next_btn = driver.find_element(By.CSS_SELECTOR, "ul.boardNav > li > a.btn_next")
                        driver.execute_script("arguments[0].click();", next_btn)
                        time.sleep(2)

                        # 페이지 증가했는지 확인
                        pagination = driver.find_element(By.CLASS_NAME, "boardPage")
                        li_list = pagination.find_elements(By.TAG_NAME, "li")
                        for li in li_list:
                            if "on" in li.get_attribute("class"):
                                page_num = int(li.text.strip())
                                break

                        if page_num == prev_page:
                            print("✅ 마지막 페이지 도달.")
                            break

                        print("➡️ 다음 버튼 클릭 (세트 이동)")
                        continue
                    except:
                        print("✅ 다음 세트 없음 (btn_next 클릭 실패).")
                        break

            except Exception as e:
                print(f"❗ 페이지 이동 중 에러: {e}")
                break

driver.quit()

# ===== 결과 저장 =====
df = pd.DataFrame(results)
df.to_csv("dongwonfnb_all_products.csv", index=False, encoding="utf-8-sig")
print("✅ 제품 목록 저장 완료. 총 항목 수:", len(df))


📄 참치/수산 > 참치캔 > 페이지 1
📄 참치/수산 > 참치캔 > 페이지 2
✅ 마지막 페이지 도달.
📄 참치/수산 > 수산캔 > 페이지 1
✅ 마지막 페이지 도달.
📄 축산 > 축산캔 > 페이지 1
✅ 마지막 페이지 도달.
📄 과일농산 > 과일농산캔 > 페이지 1
✅ 마지막 페이지 도달.
📄 즉석밥/죽/국탕찌개 > 즉석밥 > 페이지 1
✅ 마지막 페이지 도달.
📄 즉석밥/죽/국탕찌개 > 죽 > 페이지 1
✅ 마지막 페이지 도달.
📄 즉석밥/죽/국탕찌개 > 국탕찌개 > 페이지 1
✅ 마지막 페이지 도달.
📄 해조류 > 김 > 페이지 1
✅ 마지막 페이지 도달.
📄 해조류 > 미역 > 페이지 1
✅ 마지막 페이지 도달.
📄 김치 > 김치 > 페이지 1
✅ 마지막 페이지 도달.
📄 음료/샘물 > 녹차/티백 > 페이지 1
✅ 마지막 페이지 도달.
📄 음료/샘물 > 커피 > 페이지 1
✅ 마지막 페이지 도달.
📄 음료/샘물 > 샘물 > 페이지 1
✅ 마지막 페이지 도달.
📄 음료/샘물 > 기타음료 > 페이지 1
✅ 마지막 페이지 도달.
📄 냉장햄 > 냉장햄 > 페이지 1
✅ 마지막 페이지 도달.
📄 맛살/어묵 > 맛살 > 페이지 1
✅ 마지막 페이지 도달.
📄 맛살/어묵 > 어묵 > 페이지 1
✅ 마지막 페이지 도달.
📄 냉동 > 안주류 > 페이지 1
✅ 마지막 페이지 도달.
📄 냉동 > 만두 > 페이지 1
✅ 마지막 페이지 도달.
📄 냉동 > 돈까스/간식류 > 페이지 1
✅ 마지막 페이지 도달.
📄 냉동 > 완자/동그랑땡 > 페이지 1
✅ 마지막 페이지 도달.
📄 과일쨈 > 과일쨈 > 페이지 1
✅ 마지막 페이지 도달.
📄 조미/오일 > 오일 > 페이지 1
✅ 마지막 페이지 도달.
📄 조미/오일 > 조미 > 페이지 1
✅ 마지막 페이지 도달.
📄 면류 > 생/건면류 > 페이지 1
✅ 마지막 페이지 도달.
📄 분식류 > 떡볶이 > 페이지 1
✅ 마지막 페이지 도달.
📄 분식류 > 면 > 페이지 1
✅ 마지막 페이지 도달.
📄 디저트 > 디저트 > 페이지 1
✅ 마지

In [4]:
driver.find_elements(By.CSS_SELECTOR, ".boardPage li")


[<selenium.webdriver.remote.webelement.WebElement (session="571e43490ff4b1f6396e490f3c197525", element="f.463D0A388842A1B6203597A0C449AAA6.d.DC624B4057E0D37C59B64AE3FBD3C63F.e.1106")>,
 <selenium.webdriver.remote.webelement.WebElement (session="571e43490ff4b1f6396e490f3c197525", element="f.463D0A388842A1B6203597A0C449AAA6.d.DC624B4057E0D37C59B64AE3FBD3C63F.e.1108")>,
 <selenium.webdriver.remote.webelement.WebElement (session="571e43490ff4b1f6396e490f3c197525", element="f.463D0A388842A1B6203597A0C449AAA6.d.DC624B4057E0D37C59B64AE3FBD3C63F.e.1110")>,
 <selenium.webdriver.remote.webelement.WebElement (session="571e43490ff4b1f6396e490f3c197525", element="f.463D0A388842A1B6203597A0C449AAA6.d.DC624B4057E0D37C59B64AE3FBD3C63F.e.1112")>,
 <selenium.webdriver.remote.webelement.WebElement (session="571e43490ff4b1f6396e490f3c197525", element="f.463D0A388842A1B6203597A0C449AAA6.d.DC624B4057E0D37C59B64AE3FBD3C63F.e.1114")>,
 <selenium.webdriver.remote.webelement.WebElement (session="571e43490ff4b1f

In [5]:
driver.find_element(By.LINK_TEXT, "2").click()  # 또는 위에서 li → a 접근


In [45]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# ChromeDriver 경로 설정
chrome_path = "C:/Users/baby3/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe"  # 사용자 맞게 수정

# Chrome 옵션 설정 (선택)
options = Options()
options.add_argument("--start-maximized")
# options.add_argument("--headless")  # 창 없이 실행하려면 주석 해제

# 웹드라이버 실행
driver = webdriver.Chrome()
driver.get("https://www.dongwonmall.com/category/main.do?cate_id=01110065")

# "리뷰 많은순" 클릭
try:
    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "span.sort")))
    sort_buttons = driver.find_elements(By.CSS_SELECTOR, "span.sort")
    for btn in sort_buttons:
        if btn.text.strip() == "리뷰 많은순":
            btn.click()
            break
except Exception as e:
    print("리뷰 많은순 클릭 오류:", e)

# 상품 리스트 로딩 대기
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "ul#bandBestList > li.product_item"))
)
time.sleep(2)  # 동적 로딩 안정화용 잠깐 대기

# 최신 DOM에서 다시 상품 목록 수집
items = driver.find_elements(By.CSS_SELECTOR, "ul#bandBestList > li.product_item")

# 데이터 수집
product_list = []
for item in items:
    try:
        name = item.find_element(By.CSS_SELECTOR, "ul#bandBestList > li.product_item > a > p.name").text.strip()
    except:
        name = "N/A"
    try:
        price = item.find_element(By.CSS_SELECTOR, "ul#bandBestList > li.product_item > div.price_info > span.new_price > strong").text.strip()
    except:
        price = "N/A"
    try:
        grade = item.find_element(By.CSS_SELECTOR, "ul#bandBestList > li.product_item > span.star_rating > span.grade").text.strip()
    except:
        grade = "N/A"
    try:
        review = item.find_element(By.CSS_SELECTOR, "ul#bandBestList > li.product_item > span.star_rating > span.review").text.strip()
    except:
        review = "N/A"

    product_list.append({
        "상품명": name,
        "가격": price,
        "평점": grade,
        "리뷰 수": review
    })

# 브라우저 종료
driver.quit()

# 데이터프레임 출력
df = pd.DataFrame(product_list)
print(df)

# # 엑셀 저장 (선택)
df.to_excel("dongwonmall_review_sorted_products.xlsx", index=False)


                                   상품명       가격   평점      리뷰 수
0           동원참치 살코기 라이트스탠다드 85g x 20캔  31,900원    5  (2,299건)
1   동원 라이트스탠다드 참치 150gx10캔+리챔 200gx10캔  49,200원  4.9  (1,464건)
2           동원참치 살코기 라이트스탠다드 150g x 5캔  12,300원  4.9    (845건)
3          동원참치 살코기 라이트스탠다드 150g x 20캔  50,000원  4.9    (897건)
4           동원참치 살코기 라이트스탠다드 85g x 10캔  16,600원  4.9    (905건)
..                                 ...      ...  ...       ...
91                    동원 고등어 400g x 6캔  13,800원  4.8    (101건)
92                       동원참치 고추참치 85g   2,200원    5    (105건)
93      [4%적립금] 동원 고등어 400g x 24개(1박스)  45,900원    5    (121건)
94     동원 양반 소고기 미역국 460g (1~2인분) x 3개  12,000원  4.9    (101건)
95                동원 양반 단호박죽 420g x 4개  10,900원  4.8    (102건)

[96 rows x 4 columns]


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# 크롬 드라이버 경로 설정
chrome_path = "C:/Users/baby3/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe"

# 크롬 옵션
options = Options()
options.add_argument("--start-maximized")

# 드라이버 실행
driver = webdriver.Chrome()
driver.get("https://www.dongwonmall.com/category/main.do?cate_id=01110065")

# "리뷰 많은순" 클릭
WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "span.sort")))
sort_buttons = driver.find_elements(By.CSS_SELECTOR, "span.sort")
for btn in sort_buttons:
    if btn.text.strip() == "리뷰 많은순":
        btn.click()
        break

time.sleep(2)

product_list = []
current_page = 1
max_pages = 2

while True:
    print(f"[페이지 {current_page}] 수집 중...")

    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "ul#bandBestList > li.product_item"))
    )
    time.sleep(1)

    items = driver.find_elements(By.CSS_SELECTOR, "ul#bandBestList > li.product_item")
    for item in items:
        try:
            name = item.find_element(By.CSS_SELECTOR, "ul#bandBestList > li.product_item > a > p.name").text.strip()
        except:
            name = "N/A"
        try:
            price = item.find_element(By.CSS_SELECTOR, "ul#bandBestList > li.product_item > div.price_info > span.new_price > strong").text.strip()
        except:
            price = "N/A"
        try:
            grade = item.find_element(By.CSS_SELECTOR, "ul#bandBestList > li.product_item > span.star_rating > span.grade").text.strip()
        except:
            grade = "N/A"
        try:
            review = item.find_element(By.CSS_SELECTOR, "ul#bandBestList > li.product_item > span.star_rating > span.review").text.strip()
        except:
            review = "N/A"

        product_list.append({
            "페이지": current_page,
            "상품명": name,
            "가격": price,
            "평점": grade,
            "리뷰 수": review
        })

    if current_page >= max_pages:
        print("✅ 최대 페이지(10) 도달, 종료")
        break

    try:
        # 다음 페이지 번호 존재하는지 확인 (paging 내부의 ul > li > a 구조 사용)
        page_selector = f'div.paging.paging_t20r > ul > li > a'
        all_pages = driver.find_elements(By.CSS_SELECTOR, page_selector)

        next_page_number = str(current_page + 1)
        next_page_elem = None
        for p in all_pages:
            if p.text.strip() == next_page_number:
                next_page_elem = p
                break

        if next_page_elem:
            driver.execute_script("arguments[0].click();", next_page_elem)
            current_page += 1
            time.sleep(2)
        else:
            print("✅ 다음 페이지 없음, 종료")
            break
    except Exception as e:
        print("❌ 예외 발생:", e)
        break

# 저장 및 종료
driver.quit()

df = pd.DataFrame(product_list)
df.to_excel("dongwonmall_review_auto_paging.xlsx", index=False)
print("✅ 엑셀 저장 완료! 총 수집 상품 수:", len(df))


SyntaxError: invalid syntax (2659183242.py, line 7)

In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time

# 설정
chrome_path = "C:/Users/baby3/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe"
options = Options()
options.add_argument("--start-maximized")

# 실행
driver = webdriver.Chrome()
driver.get("https://www.dongwonmall.com/category/main.do?cate_id=01110067")

time.sleep(2)  # 초기 로딩 대기

# 전체 카테고리 링크 및 이름 수집
category_elements = driver.find_elements(By.CSS_SELECTOR, "div#allCateList > div.tree-group > ul.list-depth > li.list-item > a")
category_info = []

for idx, elem in enumerate(category_elements):
    try:
        name = elem.find_element(By.CSS_SELECTOR, "span.txt").text.strip()
        link = elem.get_attribute("href")
        category_info.append({"카테고리명": name, "링크": link})
    except:
        continue

# 출력 확인
for c in category_info:
    print(f"{c['카테고리명']} → {c['링크']}")
    # driver.get(c['링크'])
    # time.sleep(2)

# (선택) 첫 번째 카테고리 클릭 예시
# if category_elements:
#     print("✅ 첫 카테고리 클릭:", category_info[0]['카테고리명'])
#     driver.execute_script("arguments[0].click();", category_elements[0])
#     time.sleep(3)

# 필요한 작업 추가...

# driver.quit()


 → https://www.dongwonmall.com/category/main.do?cate_id=01110065
 → https://www.dongwonmall.com/category/main.do?cate_id=01110066
 → https://www.dongwonmall.com/category/main.do?cate_id=01110067
 → https://www.dongwonmall.com/category/main.do?cate_id=01110068
 → https://www.dongwonmall.com/category/main.do?cate_id=01110070
 → https://www.dongwonmall.com/category/main.do?cate_id=01110073
 → https://www.dongwonmall.com/category/main.do?cate_id=01110075
 → https://www.dongwonmall.com/category/main.do?cate_id=01110076
 → https://www.dongwonmall.com/category/main.do?cate_id=01110077
 → https://www.dongwonmall.com/category/main.do?cate_id=01110078
 → https://www.dongwonmall.com/category/main.do?cate_id=01110079
 → https://www.dongwonmall.com/category/main.do?cate_id=01110080
 → https://www.dongwonmall.com/category/main.do?cate_id=01110082


In [ ]:
# 출력 확인
for c in category_info:
    print(f"{c['카테고리명']} → {c['링크']}")
    driver.get(c['링크'])
    time.sleep(2)


<selenium.webdriver.remote.webelement.WebElement (session="e426788963cad073fa60bce9d573d30d", element="f.A303BDD0F71DF78C4699BEFEE2694DB7.d.76B245B905CD7D05ED710AABB87F56BE.e.231")>